# MLP Classifier

In [16]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score

In [17]:
from sklearn.neural_network import  MLPClassifier

In [18]:
# import du csv
data = pd.read_csv('csv_correction.csv')
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non


### préparation des datas

In [19]:
indexDrop = data[((data['fk_arb_etat'] == 'ABATTU') | (data['fk_arb_etat'] == 'EN PLACE') | (data['fk_arb_etat'] =='REMPLACÉ') | (data['fk_arb_etat'] == 'SUPPRIMÉ') )].index
data.drop(indexDrop, inplace = True)

In [20]:
data["fk_arb_etat"].value_counts()


fk_arb_etat
Essouché        165
Non essouché     27
Name: count, dtype: int64

In [21]:
categorical_data_cols = ['clc_quartier', 'clc_secteur', 'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation',  'fk_nomtech', 'villeca', 'feuillage']
boolean_cols = ['fk_revetement', 'remarquable']

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
dict_encoders = {}
for col in categorical_data_cols:
  ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
  ohe.fit(data[[col]])
  dict_encoders[col] = ohe
  print(type(dict_encoders[col]))
  print(col)
  ohetransform = ohe.transform(data[[col]])
  data = data.join(ohetransform)

data.drop(categorical_data_cols, axis = 1, inplace = True) 
data

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
clc_quartier
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
clc_secteur
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_stadedev
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_port
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_pied
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_situation
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
fk_nomtech
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
villeca
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
feuillage


,longitude,latitude,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,...,fk_nomtech_ROBPSE,fk_nomtech_SALBAB,fk_nomtech_SORAUC,fk_nomtech_TAXDIS,fk_nomtech_TILCOR,fk_nomtech_TILTOM,villeca_CASQ,villeca_VILLE,feuillage_Conifère,feuillage_Feuillu
4,3.304047,49.858446,5.0,2.0,170.0,Essouché,Non,40.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
73,3.270746,49.844440,12.0,3.0,160.0,Essouché,Non,50.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
76,3.270951,49.844588,13.0,4.0,155.0,Essouché,Non,50.0,10.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
92,3.294703,49.849927,24.0,9.0,245.0,Essouché,Non,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
104,3.294280,49.849615,20.0,14.0,180.0,Essouché,Non,80.0,20.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,9.0,3.0,85.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6887,3.296366,49.848250,9.0,3.0,86.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6888,3.296433,49.848229,9.0,4.0,98.0,Essouché,Non,25.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6977,3.265997,49.836176,5.0,2.0,25.0,Non essouché,Non,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [24]:
for col in boolean_cols:
  new_col = []
  for cell in data[col]:
    new_col.append(1 if cell=="Oui" else 0)
  print(col)
  print(new_col)
  data[col] = new_col

fk_revetement
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
remarquable
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### split des données

In [25]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data.drop(['fk_arb_etat'],axis = 1), data["fk_arb_etat"], test_size=0.2, random_state=42, stratify=data["fk_arb_etat"])

In [26]:
from imblearn.over_sampling import SMOTE

In [27]:
sm = SMOTE(random_state=42, sampling_strategy=0.4)
X_train, y_train = sm.fit_resample(X_train, y_train)

### create model instance

In [28]:
mlp = MLPClassifier(hidden_layer_sizes=(20, 50),validation_fraction=0.2, learning_rate="adaptive", random_state=55, max_iter=2000) # col*2, *2

### Fit

In [29]:
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              max_iter=2000, random_state=55, validation_fraction=0.2)

### Validation

In [30]:
confusion_matrix(y_train, mlp.predict(X_train), normalize="true")

array([[1., 0.],
       [0., 1.]])

## Tests

In [31]:
confusion_matrix(y_test, mlp.predict(X_test), normalize="true")

array([[0.91176471, 0.08823529],
       [0.4       , 0.6       ]])

### Export du modèle

In [45]:
import pickle

In [46]:
"""with open('mlp_model_2.pkl', 'wb') as f:
    pickle.dump(mlp, f, protocol=pickle.HIGHEST_PROTOCOL)"""

In [47]:
"""d = pd.read_json("res_df.json")"""

In [48]:
#d

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,storm_predictions
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37,EN PLACE,Jeune,semi libre,...,Alignement,Non,15,5,0,QUERUB,VILLE,Feuillu,Non,0
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160,EN PLACE,Adulte,semi libre,...,Groupe,Non,50,10,0,PINNIGnig,VILLE,Conifère,Non,0
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116,REMPLACÉ,Adulte,semi libre,...,Alignement,Non,30,10,0,ACEPSE,VILLE,Feuillu,Non,1
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150,EN PLACE,Adulte,semi libre,...,Groupe,Non,50,2,0,ACEPLA,VILLE,Feuillu,Non,1
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170,Essouché,Adulte,réduit,...,Isolé,Non,40,2,0,SALBAB,VILLE,Feuillu,Non,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,5,1,0,ALBJUL,VILLE,Feuillu,Non,0
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,3,1,0,ALBJULcho,VILLE,Feuillu,Non,0
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,5,1,0,CEDLIB,VILLE,Conifère,Non,0
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Alignement,Non,5,1,0,FAGSYLdawgol,VILLE,Feuillu,Non,0


In [49]:
"""d["storm_predictions"].value_counts()"""

storm_predictions
0    6516
1     893
Name: count, dtype: int64